### Import

In [1]:
# installing scikit-optimize library for using gp minimize 
!pip3 install scikit-optimize

     |████████████████████████████████| 81kB 2.8MB/s 


In [3]:
import warnings
warnings.filterwarnings("ignore")
from skopt import gp_minimize
from skopt.space import Real, Integer
from functools import partial
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
import xgboost as xgb

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

import os

In [12]:
# Reference: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of Dataframe is {:.3f} MB'.format(start_mem))
    
    for col in tqdm(df.columns):
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.3f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

### Creating Features

In [7]:
# Downloading data (using wget)

file_path="favorita-grocery-sales-forecasting.zip"

if not os.path.exists(file_path):
    !wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7391/44328/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1593984946&Signature=TZ8WhKQzNyAp%2B8IRIjBE3f9IPhSdR%2B8izTu2DDZLt1ZJS9M5q5pZsNpMGYYOCFwROdvxHPUf%2FIVoPslSOiRMcBdkBhumDs6xiOt9A5dzgUh6QqH3%2BzX%2F%2Be2FVjW2dg3a%2B%2FmqIwQLD7y%2B8gfRP82VlEMdGcxLLbRliMfy2ZK0BlMZgRZJ7%2BNmsdbm3V6Y%2Fk7YnIiDGH3bBopFwLN02mOhiqb96GC4gD813iLV5DRoSzegViOZjddjSBtKeNlFu86bo9oj2cjI%2BQrxQV%2F2I6IU1lKqXxkkdAl0oFzzfNUwlLForPg0nd8GMaYgdlM6Ga1liBl2QFahMYkwJUM6Hvv%2F6w%3D%3D&response-content-disposition=attachment%3B+filename%3Dfavorita-grocery-sales-forecasting.zip" -c -O 'favorita-grocery-sales-forecasting.zip'
else:
    print("File Already Present")

--2020-07-03 19:45:21--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/7391/44328/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1593984946&Signature=TZ8WhKQzNyAp%2B8IRIjBE3f9IPhSdR%2B8izTu2DDZLt1ZJS9M5q5pZsNpMGYYOCFwROdvxHPUf%2FIVoPslSOiRMcBdkBhumDs6xiOt9A5dzgUh6QqH3%2BzX%2F%2Be2FVjW2dg3a%2B%2FmqIwQLD7y%2B8gfRP82VlEMdGcxLLbRliMfy2ZK0BlMZgRZJ7%2BNmsdbm3V6Y%2Fk7YnIiDGH3bBopFwLN02mOhiqb96GC4gD813iLV5DRoSzegViOZjddjSBtKeNlFu86bo9oj2cjI%2BQrxQV%2F2I6IU1lKqXxkkdAl0oFzzfNUwlLForPg0nd8GMaYgdlM6Ga1liBl2QFahMYkwJUM6Hvv%2F6w%3D%3D&response-content-disposition=attachment%3B+filename%3Dfavorita-grocery-sales-forecasting.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480014675 (458M) [application/zip]
Saving to: ‘favorit

In [8]:
# unzipping favorita-grocery-sales-forecasting.zip

if os.path.exists('favorita-grocery-sales-forecasting.zip'):
    !unzip 'favorita-grocery-sales-forecasting.zip'
    print("File unzipped Successfully")
else:
    print("File Not Present to unzip")

Archive:  favorita-grocery-sales-forecasting.zip
  inflating: holidays_events.csv.7z  
  inflating: items.csv.7z            
  inflating: oil.csv.7z              
  inflating: sample_submission.csv.7z  
  inflating: stores.csv.7z           
  inflating: test.csv.7z             
  inflating: train.csv.7z            
  inflating: transactions.csv.7z     
File unzipped Successfully


In [9]:
#installing 7zip for extracting .7z files
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [10]:
#Extracting .7z files if they are not already extracted.

for file in os.listdir():
    if file[-3:]=='.7z':
        if os.path.exists(file[:-3]):
            print("="*50)
            print("'{}'Extracted File is Already Present".format(file[:-3]))
        elif file=='oil.csv.7z':
            !p7zip -d 'oil.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='train.csv.7z':
            !p7zip -d 'train.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='stores.csv.7z':
            !p7zip -d 'stores.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='transactions.csv.7z':
            !p7zip -d 'transactions.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='items.csv.7z':
            !p7zip -d 'items.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='holidays_events.csv.7z':
            !p7zip -d 'holidays_events.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='test.csv.7z':
            !p7zip -d 'test.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        elif file=='sample_submission.csv.7z':
            !p7zip -d 'sample_submission.csv.7z'
            print("="*50)
            print("'{}' File Extracted Successfully".format(file))

        print("="*50)



7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 474092593 bytes (453 MiB)

Extracting archive: train.csv.7z
--
Path = train.csv.7z
Type = 7z
Physical Size = 474092593
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      0% - train.csv                  1% - train.csv                  2% - train.csv                  3% - train.csv                  4% - train.csv                  5% - train.csv                  6% - train.csv                  7% - train.csv                  8% - train.cs

In [11]:
#Creating features by excecuting Pre_Processing Feature_engineering.py

exec(open('Pre_Processing Feature_engineering.py').read())

# Train Dataset Initial Date 28/6/2017
# 2 weeks

# Validation Dataset Initial Date 26/7/2017
# 1 week

# Test Dataset Initial Date 16/8/2017
# 1 week


Data Pre-Processing ...



Enter the following for Train Data :

Starting Date (Day/Month/Year) --> 28/6/2017
No. of weeks --> 2

Creating Features for data between Dates --> 2017-06-28 - 2017-07-12 (i.e. 2 weeks) 




Saving 'X_train.csv' File ...
Saving 'y_train.csv' File ...

Enter the following for Validation Data :

Starting Date (Day/Month/Year) --> 26/7/2017
No. of weeks --> 1

Creating Features for data between Dates --> 2017-07-26 - 2017-08-02 (i.e. 1 weeks) 




Saving 'X_val.csv' File ...
Saving 'y_val.csv' File ...

Enter the following for Test Data :

Starting Date (Day/Month/Year) --> 16/8/2017
No. of weeks --> 1

Creating Features for data between Dates --> 2017-08-16 - 2017-08-23 (i.e. 1 weeks) 




Saving 'X_test.csv' File ...

Saving 'sales_2017.csv' File ...
Saving 'stores_items.csv' File ...


### Reading Data

In [ ]:
# Reading X_train.csv and reducing memory usage
X_train=pd.read_csv("X_train.csv")
X_train=reduce_mem_usage(X_train)

# Reading y_train.csv and converting into numpy array
y_train = np.array(pd.read_csv( 'y_train.csv'))

Memory usage of Dataframe is 4853.989 MB



Memory usage after optimization is: 1054.381 MB
Decreased by 78.3%


In [ ]:
# Reading X_val.csv and reducing memory usage
X_val=pd.read_csv("X_val.csv")
X_val=reduce_mem_usage(X_val)

# Reading y_val.csv and converting into numpy array
y_val = np.array(pd.read_csv( 'y_val.csv'))

Memory usage of Dataframe is 808.998 MB



Memory usage after optimization is: 175.091 MB
Decreased by 78.4%


In [ ]:
# Reading X_test.csv and reducing memory usage
X_test=pd.read_csv("X_test.csv")
X_test=reduce_mem_usage(X_test)


Memory usage of Dataframe is 808.998 MB



Memory usage after optimization is: 175.730 MB
Decreased by 78.3%


In [13]:
# Reading stores_items.csv
stores_items = pd.read_csv('stores_items.csv', index_col=['store_nbr','item_nbr'])

# Reading items.csv and setting index as item_nbr
items = pd.read_csv( 'items.csv' ).set_index("item_nbr")

items = items.reindex( stores_items.index.get_level_values(1) )
items=reduce_mem_usage(items)


Memory usage of Dataframe is 5.112 MB



Memory usage after optimization is: 1.919 MB
Decreased by 62.5%


### Feature Selection

In [ ]:
# Loading Top 300 Feature Names (got by training random forest)
import pickle
with open('300_filtered_features.pkl','rb') as file:
    filtered_features = pickle.load( file)

### Defining XGBOOST

In [ ]:
def train_xgb_model(X_train,y_train,X_val,y_val,params,num_boost_rounds,n_days,items,features,X_test=None):
    '''
    Filter features from the Dataset and then
    Trains 16 different xgb models for predicting next 16 days sales . 

    Returns --> * val_pred i.e.predicted values of validation data
                * test_pred i.e.predicted values of test data if present
    '''
    params['objective'] = 'reg:squarederror'
    params['eval_metric'] = 'rmse'
    params['tree_method'] ='gpu_hist'

    val_pred = []
    test_pred = []

    #Training 16 different models for predicting next 16 days sales.
    for i in range(16):
        print("=" * 50)
        print("Step %d" % (i+1))
        print("=" * 50)

        # Filtering features
        x_train = X_train[features[i]]
        x_val = X_val[features[i]]

        #Filtering Features from test dataset if it exists.
        try:
            x_test = X_test[features[i]]
        except:
            pass

        #Creating Train Dmatrix (DMatrix is a internal data structure that used by XGBoost which is optimized for both memory efficiency and training speed.)
        dtrain = xgb.DMatrix( x_train, label=y_train[:, i],
                              weight=pd.concat([items["perishable"]] * n_days) * 0.25 + 1)
        
        #Creating Validation Dmatrix
        dval = xgb.DMatrix( x_val, label=y_val[:, i],
                            weight=items["perishable"] * 0.25 + 1)
        
        # watchist is used to see the evaluation metrics of the datasets given while training
        watchlist = [ (dtrain,'train'), (dval,'val') ]

        #Training Xgboost
        model = xgb.train(params, dtrain, num_boost_rounds, watchlist, early_stopping_rounds=125, verbose_eval=50)

        # appending results of prediction on val set
        val_pred.append(model.predict(xgb.DMatrix(x_val), ntree_limit = model.best_ntree_limit  or num_boost_rounds))

        # appending results of prediction on test set if it exists
        try:
            test_pred.append(model.predict(xgb.DMatrix(x_test), ntree_limit = model.best_ntree_limit  or num_boost_rounds))
        except:
            pass

        # Deleting unneccessary variables
        del model,dtrain,dval,x_train,x_val

    if type(X_test) != type(None):
        return val_pred,test_pred
    else:
        return val_pred


### Performance Metric


**NWRMSLE** (Normalized Weighted Root Mean Squared Logarithmic Error)

In [ ]:
def calculate_nwrmsle(true,pred,weight):
    ''' 
    Calculates Normalized Weighted Root Mean Squared Logarithmic Error (nwrmsle)

    true = true labels
    pred =  predicted labels
    weight = weights of datapoints

    returns nwrmsle '''

    temp = (true - np.array(pred).transpose())**2
    temp = temp.sum(axis=1) * weight
    nwrmsle = np.sqrt(temp.sum() / weight.sum() / 16)
    return nwrmsle

### HyperParameter Tuning (Using gp minimize)

In [ ]:

# function to fit the model and return the performance of the model
def return_model_assessment(args, X_train, y_train, X_val, y_val, model, n_days ,items , features ,num_boost_rounds =None):
    global model_hyper_params,count
    count+=1
    print('='*50)
    print("\nTraining Model No. {} ...".format(count))
    params = {model_hyper_params[i]: args[i] for i, j in enumerate(model_hyper_params)}
    print("Parameters --> ",params)


    if model=='xgb':
        val_pred = train_xgb_model(X_train,y_train,X_val,y_val,params,num_boost_rounds,n_days,items,features)

    elif model=='lgb':
        val_pred = train_lgb_model(X_train,y_train,X_val,y_val,params,num_boost_rounds,n_days,items,features)

    elif model== 'rf' :
        val_pred = train_rf_model(X_train,y_train,X_val,y_val,params,n_days,items,features)

    #calculating nwrmsle
    weight = items["perishable"] * 0.25 + 1
    nwrmsle = calculate_nwrmsle(y_val, y_pred, weight)

    return nwrmsle      #returning nwrmsle because we want to minimize it.


In [ ]:
%%time

# defining space for searching optimal parameters
space = [
    Real(0.4, 0.8, name="colsample_bytree"),
    Real(0.01, 0.5, name="gamma"),
    Real(0.001, 0.1,"log-uniform",  name="eta"),
    Integer(3, 10, name="max_depth"),
    Integer(1, 5, name="min_child_weight"),
    Real(0.4, 0.8, name="subsample"),
    ]
# Using high no. of trees (i.e. 2000) to get better model performance with early stopping.
boost_rounds = 2000

count=0
model_hyper_params = [ 'colsample_bytree', 'gamma', 'eta', 'max_depth', 'min_child_weight', 'subsample']

# Objective function which will return nwrmsle .
objective_function = partial(return_model_assessment,
                             X_train=X_train, y_train=y_train, X_val = X_val,y_val= y_val ,
                             model='xgb', n_days=2, items=items , 
                             features= filtered_features,num_boost_rounds= boost_rounds)

# Running the algorithm
n_calls = 10 # number of times to train model
results = gp_minimize(objective_function, space, base_estimator=None, n_calls=n_calls, n_random_starts=n_calls-1, random_state=42)


Streaming output truncated to the last 5000 lines.
Will train until val-rmse hasn't improved in 125 rounds.
[50]	train-rmse:0.559761	val-rmse:0.604931
[100]	train-rmse:0.544085	val-rmse:0.602832
[150]	train-rmse:0.533509	val-rmse:0.603253
[200]	train-rmse:0.524922	val-rmse:0.603916
Stopping. Best iteration:
[106]	train-rmse:0.542633	val-rmse:0.602777

Step 11
[0]	train-rmse:1.20914	val-rmse:1.20593
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 125 rounds.
[50]	train-rmse:0.573634	val-rmse:0.610328
[100]	train-rmse:0.557116	val-rmse:0.608409
[150]	train-rmse:0.546135	val-rmse:0.608551
[200]	train-rmse:0.537069	val-rmse:0.608892
Stopping. Best iteration:
[105]	train-rmse:0.555661	val-rmse:0.608264

Step 12
[0]	train-rmse:1.26137	val-rmse:1.2434
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 125 rounds.
[50]	train-rmse:0.579

In [ ]:
print("""Best parameters:
- colsample_bytree=%.6f
- gamma=%.6f
- eta=%.6f
- max_depth=%d
- min_child_weight=%d
- subsample=%.6f """ % (results.x[0], results.x[1],
                            results.x[2], results.x[3],
                            results.x[4],results.x[5]))


Best parameters:
- colsample_bytree=0.436243
- gamma=0.313009
- eta=0.005820
- max_depth=10
- min_child_weight=3
- subsample=0.743976 


In [ ]:
"Best score = %.6f" % results.fun

'Best score = 0.594137'

### Observations :


* Model --> **Xgboost**
* Best Parameters :
    * *colsample_bytree* = **0.436243**
    * *gamma* = **0.313009**
    * *eta* = **0.005820**
    * *max_depth* = **10**
    * *min_child_weight* = **3**
    * *subsample* = **0.743976** 
* Best Score (NWRMSLE) --> **0.5941**